In [1]:
"""
If you run this block more than once you will get 88 THOUSAND newlines (in the data)
"""

import json
import numpy as np
import pandas

#this code makes your editing experience using both python files 
#and notebooks a lot easier
%load_ext autoreload
#written to make things easier
%aimport utility 
%autoreload 1

#this is done to avoid the double processing problem
origin_sjc, origin_state_appeals = utility.combine_cases()
all_sjc = origin_sjc.copy(deep=True)
all_state_appeals = origin_state_appeals.copy(deep=True)


imported utility.py



In [2]:
#returns case numbers and sections in which found within a case
def find_in_text(case_data,phrase):
    cases_found = []
    for case in range(len(case_data)):
        place_found = []
        for i in range(len(case_data[case]["text"])):
            if case_data[case]["text"][i].lower().find(phrase.lower()) != -1:
                place_found.append(i)
        
        if len(place_found) > 0:
            cases_found.append([case,place_found])
    return cases_found

In [3]:
def find_in_headnote(case_data,phrase):
    cases_found = []
    for case in range(len(case_data)):
        place_found = []
        for i in range(len(case_data[case]["headnote"])):
            if case_data[case]["headnote"][i].lower().find(phrase.lower()) != -1:
                place_found.append(i)
        
        if len(place_found) > 0:
            cases_found.append([case,place_found])
    return cases_found

In [4]:
# Searches list_of_keywords in string case_data
# and returns the total number of cases that contained any keyword
#as well as the "case numbers", that is, currently
#line numbers in the file
def number_of_cases_per_keywords(case_data, list_of_keywords):
    all_cases = set()
    for keyword in list_of_keywords:
        all_with_keyword = set()
        all_in_text = find_in_text(case_data, keyword)
        all_in_headnote = find_in_headnote(case_data, keyword)
        for occurence in all_in_text:
            all_with_keyword.add(occurence[0])
        for occurence in all_in_headnote:
            all_with_keyword.add(occurence[0])
        all_cases.update(all_with_keyword)
    pretty = list(all_cases)
    pretty.sort()
    return len(pretty), pretty

In [26]:
def contains(dataframe, *keywords):
    logical_list = dataframe.text.str.contains(keywords[0])
    for i in range(1, len(keywords)):
        logical_list = logical_list | dataframe.text.str.contains(keywords[i])
    return logical_list

#elementwise xor:
#returns true at i if and only if series1[i] and series2[i]
#are different
def logic_xor(series1, series2):
    return (series1 | series2) & ~(series1 & series2)


In [6]:
important_keywords = ["confidential informant", " ci ", "snitch", "informant", "gang informant"]

In [27]:
#Something weird is going on where there's duplicate indeces: I'm not sure how to handle this
print("statistics of how we are double counting: values represent how many times an index appears")
print(all_sjc.index.to_series().value_counts().describe(), "\n\n")
      
print("showing you in the other list, max is like half of the whole list (statistics a)")
print(all_state_appeals.index.to_series().describe(), "\n")

print("statistics of how bad the problem is in that list")
print(all_state_appeals.index.to_series().value_counts().describe(), "\n\n")

print("thus, if you sort things, it looks like this")
print(all_state_appeals.sort_index()[0:6], "\n")



statistics of how we are double counting: values represent how many times an index appears
count    2009.000000
mean        1.321553
std         0.467189
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         2.000000
dtype: float64 


showing you in the other list, max is like half of the whole list (statistics a)
count    4356.000000
mean     1104.379017
std       655.543994
min         0.000000
25%       544.000000
50%      1088.500000
75%      1633.000000
max      2440.000000
dtype: float64 

statistics of how bad the problem is in that list
count    2441.000000
mean        1.784515
std         0.411243
min         1.000000
25%         2.000000
50%         2.000000
75%         2.000000
max         2.000000
dtype: float64 


thus, if you sort things, it looks like this
                                                case  \
0   STEPHANIE  & another vs. MARCIA SZABO & anoth...   
0  CITY OF CAMBRIDGE vs. EDWARD RONALD BALDASARO ...   
1  JOHN 

In [28]:
print("this leads to the following problem:")
print(all_sjc.loc[ 0, : ].text)

this leads to the following problem:
0    \nThe present case is the most recent in a ser...
0    COWIN, J. We granted the Commonwealth's applic...
Name: text, dtype: object


In [29]:
#something is seriously wrong with our algorithm somewhere in here
messed_up = all_state_appeals[1:2].text
print(messed_up.values)

print(messed_up.values[0])

['\nThis case arises from the grant of a groundwater discharge permit (permit) to the town of Falmouth (town) for the construction and operation of a wastewater treatment facility (treatment facility) and leach field.\nThe plaintiffs are abutters to the site of the proposed treatment facility or the proposed leach field, and some or all of the plaintiffs rely on private water wells near the site of the proposed treatment facility or leach field as their sole source of drinking water. Acting on the plaintiffs\' motion for judgment on the pleadings, a judge in the Superior Court upheld the decision of an acting commissioner (commissioner) of the Department of Environmental Protection (department) affirming the grant of the permit to the town.\nOn appeal, the plaintiffs argue that the commissioner erred in granting the permit because (1) the permit does not provide a reasonable margin of safety to account for any lack of knowledge\nPage 10\nconcerning the impact of the treatment facility\

In [30]:

"""
print("You see this makes things much more reasonable to remove \
newlines but it also squishes punctuation sometimes:\n")
print(messed_up.str.replace(pat="\n", repl="").iloc[0][0:500], "...\n")

print("This is a big problem in the state appeals cases. Statistics for newlines in state appeals:\n",
      all_state_appeals.text.str.count("\n").describe())

print("\nNote: there are no newlines in sjc cases: \n", all_sjc.text.str.count("\n").describe(), "\n")

#so, replace the text column to have it be good
all_state_appeals.text = all_state_appeals.text.str.replace(pat="\n", repl="")
"""


'\nprint("You see this makes things much more reasonable to remove newlines but it also squishes punctuation sometimes:\n")\nprint(messed_up.str.replace(pat="\n", repl="").iloc[0][0:500], "...\n")\n\nprint("This is a big problem in the state appeals cases. Statistics for newlines in state appeals:\n",\n      all_state_appeals.text.str.count("\n").describe())\n\nprint("\nNote: there are no newlines in sjc cases: \n", all_sjc.text.str.count("\n").describe(), "\n")\n\n#so, replace the text column to have it be good\nall_state_appeals.text = all_state_appeals.text.str.replace(pat="\n", repl="")\n'

In [32]:
vacate_indeces = all_sjc.text.str.contains("vacate") | contains(all_sjc,"vacated")


we_affirm = contains(all_sjc, "We affirm", "we affirm")
print("we affirm\n", we_affirm.value_counts())

affirmed_indeces = all_sjc.text.str.contains("affirm") | we_affirm
print("all Affirmed cases:\n", affirmed_indeces.value_counts())


we_reverse = contains(all_sjc, "we reverse") | contains(all_sjc, "We reverse")
print("\nwe reverse\n", we_reverse.value_counts())

reversed_indeces = all_sjc.text.str.contains("reverse") | we_reverse
print("\nall w reversed:\n", reversed_indeces.value_counts())


really_bad = we_affirm & we_reverse

print("\nhas both we affirm and we reverse\n", really_bad.value_counts())

both_reversed_and_affirmed = reversed_indeces & affirmed_indeces
print("\nreversed and affirmed\n", both_reversed_and_affirmed.value_counts())
print(both_reversed_and_affirmed.sort_values())
print(all_sjc.loc[ 681, : ].text)

print("\nonly one or the other appears:")
print(logic_xor(reversed_indeces, affirmed_indeces).value_counts())

we affirm
 False    1456
True     1199
Name: text, dtype: int64
all Affirmed cases:
 True     2008
False     647
Name: text, dtype: int64

we reverse
 False    2449
True      206
Name: text, dtype: int64

all w reversed:
 False    1924
True      731
Name: text, dtype: int64

has both we affirm and we reverse
 False    2601
True       54
Name: text, dtype: int64

reversed and affirmed
 False    2096
True      559
Name: text, dtype: int64
0       False
1027    False
1028    False
1029    False
1031    False
        ...  
1267     True
1270     True
1272     True
1209     True
681      True
Name: text, Length: 2655, dtype: bool
SPINA, J. The defendants, brothers John Evans (John) and Jimmy Evans (Jimmy), were convicted of murder in the first degree on theories of extreme atrocity or cruelty and of deliberate premeditation, and illegal possession of handguns (G. L. c. 269, § 10 [a]), and ammunition (G. L. c. 269, § 10 [h]). John was also convicted of assault by means of a dangerous weapon,

In [49]:
copy = all_sjc.copy(deep=True)

#need to look at vacate
def add_status(dataframe):
    
    terrible_cases = ~(contains(dataframe, "reverse", "affirm", "vacate"))
    print("Terrible cases we can't deal with, have no keywords:\n", 
         terrible_cases.value_counts(), "\n")
    
    #creating the column
    dataframe["decision"] = "undecided"
    
    definite_affirm = contains(dataframe, "We affirm", "we affirm")
    definite_reverse = contains(dataframe, "We reverse", "we reverse")
    
    dataframe.decision[definite_reverse] = "reverse"
    dataframe.decision[definite_affirm] = "affirm"
    
    undecided_still = dataframe.decision.str.contains("undecided")
    print("Decision was not that conclusive, coudn't find a \"we\" keyword :\n"
          , undecided_still.value_counts(), "\n")
    
    remaining = dataframe[undecided_still]
    print(remaining)
    particularly_inconclusive = contains(remaining, "affirm") & contains(remaining, "reverse")
    print("particularly inconclusive:\n", particularly_inconclusive.value_counts())
    
    sort_of_affirm = contains(dataframe[undecided_still], "affirm")
    sort_of_reverse = contains(dataframe[undecided_still], "reverse")
    
    dataframe.decision[sort_of_affirm] = "affirm"
    dataframe.decision[sort_of_reverse] = "reverse"
    
    undecided_still = dataframe.decision.str.contains("undecided")
    print("Decision was not that conclusive couldn't find any keyword :\n"
          , undecided_still.value_counts(), "\n")

    
    
    
add_status(copy)


Terrible cases we can't deal with, have no keywords:
 False    2343
True      312
Name: text, dtype: int64 

Decision was not that conclusive, coudn't find a "we" keyword :
 False    1351
True     1304
Name: decision, dtype: int64 

                                                   case  \
0                  COMMONWEALTH  vs.  ADMILSON RESENDE.   
2                LINDA S. BOWERS  vs.  P. WILE'S, INC.    
4                    COMMONWEALTH  vs.  ROBERT D. WADE.   
9      CURT F. PFANNENSTIEHL  vs.  DIANE L. PFANNENS...   
10                        COMMONWEALTH  vs.  KIM HENRY.   
...                                                 ...   
1997  JON BUTCHER vs. UNIVERSITY OF MASSACHUSETTS & ...   
1999                            COMMONWEALTH vs. ERVIN    
2000                       COMMONWEALTH vs. FREDERICK H   
2003                      COMMONWEALTH vs. RICHARD S. F   
2005                             KAROL E. SIMONTON, pet   

                                               headnote  \

ValueError: cannot reindex from a duplicate axis

### searching for total number

In [ ]:
print("total number of cases was: ", len(all_sjc) + len(all_state_appeals))

In [ ]:
result = number_of_cases_per_keywords(cases, important_keywords)
print(result)

In [ ]:
print(number_of_cases_per_keywords(appeals, important_keywords))

In [ ]:
important_keywords.remove("informant")
print(number_of_cases_per_keywords(cases, important_keywords)[0] + number_of_cases_per_keywords(cases, important_keywords)[0])

### Searching terms in text then headnote for cases.csv

In [ ]:
print(find_in_text(cases,"confidential informant"))

In [ ]:
print(find_in_headnote(cases,"confidential informant"))

In [ ]:
print(find_in_text(cases," ci "))

In [ ]:
print(find_in_headnote(cases," ci "))

In [ ]:
print(find_in_text(cases,"snitch"))

In [ ]:
print(find_in_headnote(cases,"snitch"))

In [ ]:
print(find_in_text(cases,"informant"))

In [ ]:
print(find_in_headnote(cases,"informant"))

In [ ]:
print(find_in_text(cases,"gang informant"))

In [ ]:
print(find_in_headnote(cases,"gang informant"))

In [ ]:
print(find_in_text(cases,"mafia informant"))

In [ ]:
print(find_in_headnote(cases,"mafia informant"))

In [ ]:
print(find_in_text(cases,"organized crime informant"))

In [ ]:
print(find_in_headnote(cases,"organized crime informant"))

In [ ]:
print(find_in_text(cases,"Stephen Lynch informant"))

In [ ]:
print(find_in_headnote(cases,"Stephen Lynch informant"))

In [ ]:
print(find_in_text(cases,"federal prison informant"))

In [ ]:
print(find_in_headnote(cases,"federal prison informant"))

In [ ]:
print(find_in_text(cases,"prison informant"))

In [ ]:
print(find_in_headnote(cases,"prison informant"))

In [ ]:
print(find_in_text(cases,"Whitey Bulger informant"))

In [ ]:
print(find_in_headnote(cases,"Whitey Bulger informant"))

In [ ]:
print(find_in_text(cases,"U.S. government informant"))

In [ ]:
print(find_in_headnote(cases,"U.S. government informant"))

In [ ]:
print(find_in_text(cases,"government informant"))

In [ ]:
print(find_in_headnote(cases,"government informant"))

In [ ]:
print(find_in_text(cases,"government snitch"))

In [ ]:
print(find_in_headnote(cases,"government snitch"))

### Searching terms in text then headnote for cases_appeals.csv

In [ ]:
print(find_in_text(appeals,"confidential informant"))

In [ ]:
print(find_in_headnote(appeals,"confidential informant"))

In [ ]:
print(find_in_text(appeals," ci "))

In [ ]:
print(find_in_headnote(appeals," ci "))

In [ ]:
print(find_in_text(appeals,"snitch"))

In [ ]:
print(find_in_headnote(appeals,"snitch"))

In [ ]:
print(find_in_text(appeals,"informant"))

In [ ]:
print(find_in_headnote(appeals,"informant"))

In [ ]:
print(find_in_text(appeals,"gang informant"))

In [ ]:
print(find_in_headnote(appeals,"gang informant"))

In [ ]:
print(find_in_text(appeals,"mafia informant"))

In [ ]:
print(find_in_headnote(appeals,"mafia informant"))

In [ ]:
print(find_in_text(appeals,"organized crime informant"))

In [ ]:
print(find_in_headnote(appeals,"organized crime informant"))

In [ ]:
print(find_in_text(appeals,"Stephen Lynch informant"))

In [ ]:
print(find_in_headnote(appeals,"Stephen Lynch informant"))

In [ ]:
print(find_in_text(appeals,"federal prison informant"))

In [ ]:
print(find_in_headnote(appeals,"federal prison informant"))

In [ ]:
print(find_in_text(appeals,"prison informant"))

In [ ]:
print(find_in_headnote(appeals,"prison informant"))

In [ ]:
print(find_in_text(appeals,"Whitey Bulger informant"))

In [ ]:
print(find_in_headnote(appeals,"Whitey Bulger informant"))

In [ ]:
print(find_in_text(appeals,"U.S. government informant"))

In [ ]:
print(find_in_headnote(appeals,"U.S. government informant"))

In [ ]:
print(find_in_text(appeals,"government informant"))

In [ ]:
print(find_in_headnote(appeals,"government informant"))

In [ ]:
print(find_in_text(appeals,"government snitch"))

In [ ]:
print(find_in_headnote(appeals,"government snitch"))

In [ ]:
print(len(find_in_text(cases,"appeal")))

In [ ]:
len(cases)